<a href="https://colab.research.google.com/github/gimpapa/Data_analysis_that_can_be_used_right_in_the_field_with_Python/blob/main/SentiWord(Korean).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Korean Emotional Dictionary by KNU

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [4]:
import pandas as pd

In [5]:
senti_dic=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/SentiWord_Dict.txt',sep='\t',encoding='utf-8',header=None)

In [ ]:
senti_dic.head()

In [8]:
senti_dic.columns=['word','score']

In [ ]:
senti_dic.head()

In [ ]:
senti_dic.tail()

In [ ]:
senti_dic[senti_dic['score'].isnull()]

In [ ]:
senti_dic.iloc[14850]

In [16]:
senti_dic.loc[14850,'word']='갈등'
senti_dic.loc[14850,'score']=-1

In [ ]:
senti_dic.iloc[14850]

In [ ]:
senti_dic.isnull().sum()

# Open a txt file

In [20]:
txt=open('/content/gdrive/My Drive/Colab Notebooks/review_word_cloud.txt').readlines()

In [ ]:
txt[0:5]

# CountVectorizer()

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
vector=CountVectorizer()

In [25]:
bow=vector.fit_transform(txt)

In [ ]:
term=vector.get_feature_names()

In [27]:
import pandas as pd

In [28]:
dtm=pd.DataFrame.sparse.from_spmatrix(data=bow,columns=term)

In [ ]:
dtm.shape

In [ ]:
dtm

# Write code that can evaluate sentiment scores.
- If the word is in the sentiment dictionary, take the score and multiply it by the count value.

In [31]:
ds=[]
for i in range(dtm.shape[0]):
    d=[]
    for j in range(dtm.shape[1]):
        if dtm.iloc[i,j]>=1:
            if senti_dic.loc[senti_dic.word==dtm.columns[j],'score'].empty!=True:
                d.append(senti_dic.loc[senti_dic.word==dtm.columns[j],'score'].values[0]*dtm.iloc[i,j])
            else:
                d.append(0)
        else:
            d.append(0)
    s=sum(d)
    ds.append(s)

In [ ]:
ds[0:5]

In [ ]:
res=pd.DataFrame(list(zip(txt,ds)),columns=['doc','score'])
res.head()

# A review score
- greater than 0 is positive,
- 0 is neutral,
- less than 0 is negative.

In [34]:
pn=[]
for row in res['score']:
    if row>0:
        pn.append('Positive')
    elif row==0:
        pn.append('Neutral')
    else:
        pn.append('Negative')

In [ ]:
res['pn']=pn
res.head()

# Grouping results by count

In [ ]:
res_g=res.groupby([pn]).count()
res_g

In [37]:
pct=res_g['pn']/res_g['pn'].sum()

In [39]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.pie(pct,labels=res_g.index,autopct='%.1f%%',startangle=0,counterclock=False)
plt.show()